In [1]:
import os
os.chdir('/home/blueconet/Downloads/Image Classification')

In [2]:
from cifar_input import cifar10_input
import ReSingNet_cifar10 as model
import tensorflow as tf

BATCH_SIZE=16
EVAL_SIZE=1000
NUM_GPUS=8

def tower_loss(batch, scope):
    images, labels = batch
    
    logits = model.inference(images, keep_prob)
    _ = model.loss(logits, labels)
    
    losses = tf.get_collection('losses', scope)
    
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

def average_grads(list_grads):
    grads = [(tf.reduce_mean([list_grads[j][i][0] for j in range(len(list_grads))],
                             0),
             list_grads[0][i][1]) for i in range(len(list_grads[0]))]
    return grads

In [6]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
    train_data, test_data = cifar10_input()
    
    train_batch = train_data.dist_batch(BATCH_SIZE,20000)
    test_batch = test_data.batch(200)

    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y_ = tf.placeholder(tf.int32, [x.get_shape()[0]])
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.placeholder(tf.float32)
    opt = tf.train.GradientDescentOptimizer(lr)

    with tf.device('/gpu:0'):
        logit = model.inference(x, keep_prob)
        loss = model.loss(logit, y_)
        grads = opt.compute_gradients(loss)    
    
    train_op = opt.apply_gradients(grads)
    sess = tf.Session()
    
    rate = 0.001
    
    sess.run(tf.initialize_all_variables())
    tf.train.start_queue_runners(sess=sess)
    for i in range(20):
        image_batch, label_batch = sess.run(train_batch)
        _, cross_entropy = sess.run([train_op, loss], feed_dict = {x: image_batch, y_: label_batch, keep_prob: 0.5, lr:rate})
        print("step %d, cross_entropy %g"%(i, cross_entropy))

step 0, cross_entropy 78.7641
step 1, cross_entropy 78.4237
step 2, cross_entropy 77.8237
step 3, cross_entropy 77.2804
step 4, cross_entropy 76.6903
step 5, cross_entropy 76.9514
step 6, cross_entropy 76.7231
step 7, cross_entropy 76.6587
step 8, cross_entropy 76.6112
step 9, cross_entropy 76.8441
step 10, cross_entropy 76.3099
step 11, cross_entropy 76.9578
step 12, cross_entropy 76.6444
step 13, cross_entropy 76.8703
step 14, cross_entropy 76.5216
step 15, cross_entropy 76.7703
step 16, cross_entropy 76.5562
step 17, cross_entropy 76.7892
step 18, cross_entropy 76.7734
step 19, cross_entropy 76.6043


In [4]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)

        with tf.device('/gpu:0'):
            with tf.name_scope('%s_%d' % ('tower', 0)) as scope:
                loss = tower_loss(train_batch, scope)
                grads = opt.compute_gradients(loss)    
    
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        mean_loss = 0.
        least = 10.
        count = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean_loss = mean_loss + cross_entropy/50
    
            if i%50 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, mean_loss))
      
                if mean_loss > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean_loss
                mean_loss=0
        
            if i%250 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                print("test accuracy %g"%(eval_accuracy))

step 50, cross_entropy nan
step 100, cross_entropy nan
step 150, cross_entropy nan
step 200, cross_entropy nan
step 250, cross_entropy nan
test accuracy 0
step 300, cross_entropy nan


KeyboardInterrupt: 

In [7]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)
        
        tower_grads = []
        tower_losses = []
        
        for i in range(NUM_GPUS):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                    loss = tower_loss(train_batch, scope)
                    grads = opt.compute_gradients(loss)
                    tower_losses.append(loss)
                    tower_grads.append(grads)
                    
        grads = average_grads(tower_grads)
        mean_loss = tf.reduce_mean(tower_losses)
  
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.01
        least = 10.
        mean = 0.
        count = 0.
        highest = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))
      
                if mean > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean
                
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
        print("highest accuracy: %g"%(highest))

step 199, cross_entropy 65.4808
test accuracy 0.208
step 399, cross_entropy 56.2519
test accuracy 0.264
step 599, cross_entropy 47.9626
test accuracy 0.229
step 799, cross_entropy 41.1196
test accuracy 0.257
step 999, cross_entropy 35.3608
test accuracy 0.306
step 1199, cross_entropy 30.2464
test accuracy 0.301
step 1399, cross_entropy 26.1215
test accuracy 0.283
step 1599, cross_entropy 22.8776
test accuracy 0.278
step 1799, cross_entropy 19.1994
test accuracy 0.326
step 1999, cross_entropy 17.3684
test accuracy 0.31
step 2199, cross_entropy 14.7481
test accuracy 0.321
step 2399, cross_entropy 12.5449
test accuracy 0.321
step 2599, cross_entropy 11.1675
test accuracy 0.301
step 2799, cross_entropy 10.2266
test accuracy 0.295
step 2999, cross_entropy 9.02957
test accuracy 0.331
step 3199, cross_entropy 7.72984
test accuracy 0.332
step 3399, cross_entropy 6.84819
test accuracy 0.301
step 3599, cross_entropy 5.79049
test accuracy 0.308
step 3799, cross_entropy 5.17669
test accuracy 0.333

In [8]:
rate = 0.002
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 2.15845
test accuracy 0.406
step 399, cross_entropy 1.53343
test accuracy 0.396
step 599, cross_entropy 2.03566
test accuracy 0.397
step 799, cross_entropy 1.43554
test accuracy 0.459
step 999, cross_entropy 1.57236
test accuracy 0.463
step 1199, cross_entropy 1.32138
test accuracy 0.439
step 1399, cross_entropy 1.6097
test accuracy 0.413
step 1599, cross_entropy 1.69142
test accuracy 0.457
step 1799, cross_entropy 1.47056
test accuracy 0.416
step 1999, cross_entropy 1.47048
test accuracy 0.463
step 2199, cross_entropy 1.98591
test accuracy 0.412
step 2399, cross_entropy 1.39062
test accuracy 0.448
step 2599, cross_entropy 1.95316
test accuracy 0.457
step 2799, cross_entropy 1.89165
test accuracy 0.439
step 2999, cross_entropy 1.57108
test accuracy 0.471
step 3199, cross_entropy 1.92245
test accuracy 0.421
step 3399, cross_entropy 2.00795
test accuracy 0.441
step 3599, cross_entropy 1.73291
test accuracy 0.412
step 3799, cross_entropy 1.49138
test accuracy 0.427

In [9]:
rate = 0.0004
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 1.88989
test accuracy 0.469
step 399, cross_entropy 1.48329
test accuracy 0.463
step 599, cross_entropy 1.36201
test accuracy 0.439
step 799, cross_entropy 1.37482
test accuracy 0.472
step 999, cross_entropy 1.44761
test accuracy 0.487
step 1199, cross_entropy 1.98549
test accuracy 0.477
step 1399, cross_entropy 1.26135
test accuracy 0.494
step 1599, cross_entropy 1.78201
test accuracy 0.464
step 1799, cross_entropy 1.51874
test accuracy 0.47
step 1999, cross_entropy 1.40021
test accuracy 0.496
step 2199, cross_entropy 1.43184
test accuracy 0.484
step 2399, cross_entropy 1.78317
test accuracy 0.445
step 2599, cross_entropy 2.00336
test accuracy 0.467
step 2799, cross_entropy 1.22974
test accuracy 0.457
step 2999, cross_entropy 1.60837
test accuracy 0.486
step 3199, cross_entropy 1.03278
test accuracy 0.444
step 3399, cross_entropy 1.98058
test accuracy 0.461
step 3599, cross_entropy 1.59977
test accuracy 0.453
step 3799, cross_entropy 1.77354
test accuracy 0.493